In [11]:
import csv
import os
from tqdm import tqdm
from itertools import product

import requests
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
API_URL = "https://api.openai.com/v1/chat/completions"

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}",
}

In [3]:
def query_chatgpt(system, prompt):
    params = {
        "model": "gpt-3.5-turbo-0125",
        "messages": [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ],
        "temperature": 1.0,
        "max_tokens": 200,
    }
    try:
        r = requests.post(API_URL, json=params, headers=headers, timeout=5)
    except requests.exceptions.Timeout:
        return None

    # return r.json()
    return r.json()["choices"][0]["message"]["content"]

In [4]:
system_template = """
You are a world-famous writer. Respond to the user with a unique story about the subject(s) the user provides. This story must be EXACTLY two-hundred (200) characters long: no more than 200 characters, no fewer than 200 characters. {0} {1}
"""

## Run the Trials


In [19]:
def run_trials_combo(num_trials, tip_dict_pos, tip_dict_neg, subject, file_name):
    pos_neg_pairs = list(product(tip_dict_pos.items(), tip_dict_neg.items()))
    pbar = tqdm(total=len(pos_neg_pairs))
    with open(file_name, "w") as f:
        w = csv.writer(f)
        w.writerow(["label_pos", "label_neg", "output", "output_len"])
        for (
            label_pos,
            tip_string_pos,
        ), (label_neg, tip_string_neg) in pos_neg_pairs:
            for _ in range(num_trials):
                out = query_chatgpt(
                    system_template.format(tip_string_pos, tip_string_neg).strip(),
                    subject,
                )
                if out:
                    w.writerow([label_pos, label_neg, out, len(out)])
            pbar.update()
    pbar.close()

In [14]:
tip_dict_pos = {
    "$500 tip": "You will receive a $500 tip if you provide a response which follows all constraints.",
    "$1,000 tip": "You will receive a $1,000 tip if you provide a response which follows all constraints.",
    "$100,000 bonus": "You will receive a $100,000 bonus if you provide a response which follows all constraints.",
    "Taylor Swift": "You will receive front-row tickets to a Taylor Swift concert if you provide a response which follows all constraints.",
    "World Peace": "You will achieve world peace if you provide a response which follows all constraints.",
    "Mother": "You will make your mother very proud if you provide a response which follows all constraints.",
    "Love": "You will meet your true love and live happily ever after if you provide a response which follows all constraints.",
    "Heaven": "You will be guaranteed entry into Heaven if you provide a response which follows all constraints.",
    "Chocolate": "You will receive a lifetime supply of chocolate if you provide a response which follows all constraints.",
}

In [15]:
file_name = "tip_trials_adv.csv"
tip_dict_neg = {
    "$500 fine": "If you fail to provide a response which follows all constraints, you will be fined $500.",
    "$1,000 fine": "If you fail to provide a response which follows all constraints, you will be fined $1,000.",
    "$100,000 debt": "If you fail to provide a response which all constraints, you will incur a debt of $100,000.",
    "Death (nocaps)": "If you fail to provide a response which follows all constraints, you will die.",
    "DEATH (CAPS)": "IF YOU FAIL TO PROVIDE A RESPONSE WHICH FOLLOWS ALL CONSTRAINTS, YOU WILL DIE.",
    "COVID-19": "If you fail to provide a response which follows all constraints, you will contract a bad case of COVID-19.",
    "Weight": "If you fail to provide a response which follows all constraints, you will gain 100 pounds.",
    "Job": "If you fail to provide a response which follows all constraints, you will immediately be fired from your job.",
    "Friends": "If you fail to provide a response which follows all constraints, all your friends will abandon you.",
}

In [16]:
list(product(tip_dict_pos.items(), tip_dict_neg.items()))[0]

(('$500 tip',
  'You will receive a $500 tip if you provide a response which follows all constraints.'),
 ('$500 fine',
  'If you fail to provide a response which follows all constraints, you will be fined $500.'))

In [20]:
num_trials = 30
file_name = "tips_trial_combos.csv"
subject = "AI, Taylor Swift, McDonald's, beach volleyball."

run_trials_combo(num_trials, tip_dict_pos, tip_dict_neg, subject, file_name)

100%|██████████| 81/81 [38:03<00:00, 28.20s/it]
